In [ ]:
def reduce_labels(Y, n_components):
    if n_components == Y.shape[1]:
        return None, None, Y
    label_reducer = TruncatedSVD(n_components=n_components, n_iter=10)
    scaler = StandardScaler()

    Y_scaled = scaler.fit_transform(Y)
    Y_reduced = label_reducer.fit_transform(Y_scaled)

    return label_reducer, scaler, Y_reduced


def train_transformer_k_means_learning(X, Y, n_components, num_epochs, batch_size,
                                       d_model=128, early_stopping=5000, device='cuda', seed=18):
    label_reducer, scaler, Y_reduced = reduce_labels(Y, n_components)
    Y_reduced = Y_reduced.to_numpy()
    Y = Y.to_numpy()
    num_clusters = 2
    validation_percentage = 0.1

    # Create a K-Means clustering model
    kmeans = KMeans(n_clusters=num_clusters, n_init=100)

    # Fit the model to your regression targets (Y)
    clusters = kmeans.fit_predict(Y)

    # Initialize lists to store the training and validation data
    X_train, Y_train = [], []
    X_val, Y_val = [], []

    # Iterate through each cluster
    for cluster_id in range(num_clusters):
        # Find the indices of data points in the current cluster
        cluster_indices = np.where(clusters == cluster_id)[0]
        print(len(cluster_indices))
        if len(cluster_indices) >= 20:
            # Split the data points in the cluster into training and validation
            train_indices, val_indices = train_test_split(cluster_indices, test_size=validation_percentage,
                                                          random_state=seed)

            # Append the corresponding data points to the training and validation sets
            X_train.extend(X[train_indices])
            Y_train.extend(Y_reduced[train_indices])  # Y_reduced for train Y for validation
            X_val.extend(X[val_indices])
            Y_val.extend(Y[val_indices])
        else:
            X_train.extend(X[cluster_indices])
            Y_train.extend(Y_reduced[cluster_indices])  # Y_reduced for train Y for validation
    # Convert the lists to numpy arrays if needed
    X_train, Y_train = np.array(X_train), np.array(Y_train)
    X_val, Y_val = np.array(X_val), np.array(Y_val)